Wheat Detection - Kaggle Competition

Load Yolov5

In [ ]:
!git clone https://github.com/samthomp/yolov5 # clone repo with customized wheat edits

Cloning into 'yolov5'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 834 (delta 14), reused 8 (delta 3), pack-reused 804
Receiving objects: 100% (834/834), 3.39 MiB | 760.00 KiB/s, done.
Resolving deltas: 100% (544/544), done.


In [ ]:
%cd yolov5
!pip install -U -r requirements.txt

/content/yolov5
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-9znpddga
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-9znpddga
Requirement already up-to-date: Cython in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (0.29.20)
     |████████████████████████████████| 20.4MB 7.9MB/s 
     |████████████████████████████████| 28.2MB 78kB/s 
Requirement already up-to-date: torch>=1.4 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 5)) (1.5.1+cu101)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 6)) (3.2.2)
     |████████████████████████████████| 2.1MB 51.8MB/s 
Requirement already up-to-date: tensorboard in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 8)) (2.2.2)
     |████████████████████████████████| 276kB 51.3MB/s 
Requirement already up-to-date: torchvision in /usr/lo

Split CSV into multiple Data Files

In [ ]:
# download dataset
!python3 -c "from yolov5.utils.google_utils import gdrive_download; gdrive_download('1rRTfBlMx_3DDXPA9hfcw9EAR6PLQi2d3','wheat.zip')" 


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2443      0 --:--:-- --:--:-- --:--:--  2457
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  607M    0  607M    0     0  74.6M      0 --:--:--  0:00:08 --:--:-- 92.4M


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# file and folder packages
import glob
import os
from os import path

# image manipulation related imports
import shutil
import cv2
from PIL import Image
from skimage import data
from skimage import color
from skimage import img_as_float

In [ ]:
import math
import random
import subprocess
import time
from copy import copy
from pathlib import Path
from sys import platform

import torch
import torch.nn as nn
import torchvision
import yaml
from scipy.signal import butter, filtfilt
from tqdm import tqdm

In [ ]:
%cd wheat
!ls

/content/wheat
images	labels	train.csv


In [ ]:
def resize_img (img):
    
    scale_percent = 0.5 # percent of original size
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    dim = (width, height)
    
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    return resized

In [ ]:
img_sze = 512

# add some noise to training data
def gauss_blur(image, level):
    return cv2.blur(image, (level * 2 + 1, level * 2 + 1))

def gauss_noise(image):
    
    for i in range(image.shape[2]):
        c = image[:, :, i]
        diff = 255 - c.max();
        noise = np.random.normal(0, np.random.randint(1, 6), c.shape)
        noise = (noise - noise.min()) / (noise.max() - noise.min())
        noise = diff * noise
        image[:, :, i] = c + noise.astype(np.uint8)
        
    return image

def constrast_img (image):
    
    alpha = 1.15 # Simple contrast control 1.0-3.0]
    beta = 1    # Simple brightness control  [0-100]
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)     

def green_scale (image):

    filter_img  = np.full((img_sze, img_sze, 3), (143, 188, 143), np.uint8)
    return cv2.addWeighted(image, 0.8, filter_img, 0.2, 0)
    
def yellow_scale  (image):
    
    filter_img  = np.full((img_sze, img_sze, 3), (250, 250, 205), np.uint8)
    return cv2.addWeighted(image, 0.85, filter_img, 0.15, 0)

In [ ]:
# Apply combinations of random transformations to images
def transform_image(img, select):
        
    # recolor & tint
    if (select[0] > 0):        
        img = constrast_img (img)
    if (select[1] > 0) :
        img = green_scale (img)        
    if (select[2] > 0):
        img = yellow_scale (img)                           
    
    # blur & noise
    if (select[4] > 0):
        img = gauss_blur(img, 2)
    if (select[5]) > 0: 
        img = gauss_noise(img)
        
    if (select[3] > 0):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    return img

## make sure to rename the preprocess and preprocess-test folder

In [ ]:
# Split data randomly into training and test set
def preprocess_trainingdata ():
    
    n = 700    
    count = 0
    np.random.seed(42)
    
    all_imgs = glob.glob("images/preprocess/*.jpg")
    all_imgs = [i.split("/")[-1].replace(".jpg", "") for i in all_imgs]
    np.random.shuffle(all_imgs)
    test_set = all_imgs #[-n:]
    
    for image_names in test_set:
                
        select = np.random.randint(2, size=(3422, 6))
        
        # read the pre-process images, transform then and then save them to the train
        img_filename = "images/preprocess/{}.jpg".format(image_names)
        img = cv2.imread(img_filename, cv2.IMREAD_UNCHANGED)
        
        # resize the image
        img = resize_img (img)
        
        #  save resized original
        img_filename = "images/train/{}-0.jpg".format(image_names)
        cv2.imwrite(img_filename, img)        
                
        img_r = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        img_filename = "images/train/{}-r.jpg".format(image_names)
        cv2.imwrite(img_filename, img_r)
        
        # transform resized image
        img = transform_image(img, select[count]) 
        
        # rotate image 180 degrees
        img = cv2.rotate(img, cv2.ROTATE_180)
        img_filename = "images/train/{}-i.jpg".format(image_names)
        cv2.imwrite(img_filename, img)
                        
        count = count + 1
        
    return count

preprocess_trainingdata ()

3422

In [ ]:
# Add features to test images
def preprocess_testdata ():
    
    #n = 600
    count = 0
    np.random.seed(42)
        
    all_imgs = glob.glob("images/preprocess-test/*.jpg")
    all_imgs = [i.split("/")[-1].replace(".jpg", "") for i in all_imgs]

    # copy the original images before the transformation
    for image_names in all_imgs:        
        shutil.copy2("images/preprocess-test/{}.jpg".format(image_names), 'images/test/')
    
    for image_names in all_imgs:
        
        # save the transformed images in the train folder
        img_filename = "images/test/{}.jpg".format(image_names)
        img = cv2.imread(img_filename, cv2.IMREAD_UNCHANGED)
        count = count + 1
                
        #img = transform_image(img)           
        img = resize_img (img)
        #img = constrast_img(img)        
        cv2.imwrite(img_filename, img) 
        
    return True

preprocess_testdata ()

True

In [ ]:
# test different filters to bring near test set closer to the baseline
def coord_rotate(x, y, xm, ym, a):
    
    cos = math.cos
    sin = math.sin

    #a = a * math.pi / 180 #onvert to radians because that is what python likes

    # Subtract midpoints, so that midpoint is translated to origin
    # and add it in the end again
    xr = (x - xm) * cos(a) - (y - ym) * sin(a)   + xm
    yr = (x - xm) * sin(a) + (y - ym) * cos(a)   + ym

    return [xr, yr]

Label Processing

In [ ]:
boxes_df = pd.read_csv('train.csv')

In [ ]:
def calculate_inverted (yolo_box):
    
    xarr = yolo_box.strip('[]').split(",") #.astype(float)
    xarr = np.array(xarr)
    bbox = xarr.astype(float)
    
    new_coords = coord_rotate(bbox[0], bbox[1], 0.5, 0.5, math.radians(180))
    bbox[0] = new_coords[0]
    bbox[1] = new_coords[1]
         
    return str([bbox[0], bbox[1], bbox[2], bbox[3]])

## rotate yolo points
def calculate_rotated (yolo_box):
    
    xarr = yolo_box.strip('[]').split(",") #.astype(float)
    xarr = np.array(xarr)
    bbox = xarr.astype(float)

    new_coords = coord_rotate(bbox[0], bbox[1], 0.5, 0.5, math.radians(90))    
    
    bbox[0] = new_coords[0]
    bbox[1] = new_coords[1]
    xtemp = bbox[2]
    bbox[2] = bbox[3]
    bbox[3] = xtemp    
    
    return str([bbox[0], bbox[1], bbox[2], bbox[3]])

## External code pabloberhauser kaggle
def convert(size, box):
    
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def convert_to_yolo_label(coco_format_box, w = 1024, h = 1024):

    xarr = coco_format_box.strip('[]').split(",") #.astype(float)
    xarr = np.array(xarr)
    bbox = xarr.astype(float)
    
    xmin = bbox[0]
    xmax = bbox[0] + bbox[2]
    ymin = bbox[1]
    ymax = bbox[1] + bbox[3]
    b = (float(xmin), float(xmax), float(ymin), float(ymax))
    yolo_box = convert((w, h), b)
  
    # Sanity check on calculations - Take this opportunity to check that conversion works
    if np.max(yolo_box) > 1 or np.min(yolo_box) < 0: 
        print("BOX HAS AN ISSUE")
        
    return str(yolo_box)

# convert the bbox to the darknet format
boxes_df['class'] = 0
boxes_df['bbox_cords'] = 0
boxes_df['yolo_box'] = boxes_df.bbox.apply(convert_to_yolo_label)
boxes_df['yolo_box_i'] = boxes_df.yolo_box.apply(calculate_inverted)
boxes_df['yolo_box_r'] = boxes_df.yolo_box.apply(calculate_rotated)

In [ ]:
print(boxes_df.head(3))

    image_id  ...                                         yolo_box_r
0  b6ab77fd7  ...     [0.765625, 0.841796875, 0.03515625, 0.0546875]
1  b6ab77fd7  ...  [0.4365234375, 0.2841796875, 0.056640625, 0.12...
2  b6ab77fd7  ...     [0.4296875, 0.404296875, 0.15625, 0.072265625]

[3 rows x 10 columns]


In [ ]:
print("We have {} unique images with boxes.".format(len(boxes_df.image_id.unique())))
unique_img_ids = boxes_df.image_id.unique()

We have 3373 unique images with boxes.


In [ ]:
# write the label files in the format that Yolo expects - darknet format
images_path = "images/train/"
labels_path = "labels/train/"

def delete_txtlabels ():

    image_names = boxes_df['image_id'].unique()
    for name in image_names:
        fpath = labels_path + name + '-0.txt'    
        os.system("rm " + fpath)

    return

#delete_txtlabels()

def write_bboxfiles ():

    image_names = boxes_df['image_id'].unique()
    
    img_ext = ['-0','-r','-i'] #
    src_col = ['yolo_box','yolo_box_r','yolo_box_i'] # 
    
    full_cols = ['class', 'x_center', 'y_center', 'bb_width', 'bb_height']
    expanded_cols = ['x_center', 'y_center', 'bb_width', 'bb_height']
    
    for i in range(len(img_ext)):        
                
        boxes_df['bbox_cords'] = boxes_df[src_col[i]].str.strip('[]')
        boxes_df[expanded_cols] = boxes_df['bbox_cords'].str.split(",", expand=True).astype(float)
        
        for name in image_names:

            fpath = labels_path  + name + img_ext[i] + '.txt'
            image = images_path + name + img_ext[i] + '.jpg'

            if (os.path.isfile(image)):
                # get the rows in the dataframe that correspond to this image
                temp_df = boxes_df[boxes_df['image_id'] == name]
                temp_df[full_cols].to_csv(fpath, header=None, index=None, sep=' ')                    
                
    return 

write_bboxfiles ()

In [ ]:
def getimages_wnolabels ():
    
    all_imgs = glob.glob("images/train/*-0.jpg")
    all_imgs = [i.split("/")[-1].replace("-0.jpg", "") for i in all_imgs]

    positive_imgs = boxes_df.image_id.unique()

    img_list = set(all_imgs)

    # find all of the images without labels
    img_list.difference_update(positive_imgs)
    print(len(all_imgs), len(positive_imgs), len(img_list))

    nolabel_images = list(img_list)
    
    return nolabel_images

def create_emptyLabels():
    
    negative_images = getimages_wnolabels ()
    
    img_ext = ['0','r','i'] #    

    for i in range(len(img_ext)): 
        
        # write an empty label file for images with no bboxes
        for image in list(negative_images):            
            fname = "{}-".format(image) + img_ext[i] + ".txt"
            fpath = "labels/train/"
            with open(os.path.join( fpath, fname), 'w') as fp: 
                pass
    
    return 

create_emptyLabels()

3422 3373 49


In [ ]:
# Split data randomly into training and test set
def split_train_testset ():
    n = 610    

    all_imgs = glob.glob("images/train/*-0.jpg")
    all_imgs = [i.split("/")[-1].replace(".jpg", "") for i in all_imgs]
    np.random.shuffle(all_imgs)
    test_set = all_imgs[-n:]

    for image_labels in test_set:
        shutil.move("images/train/{}.jpg".format(image_labels), 'images/val/')
        shutil.move("labels/train/{}.txt".format(image_labels), 'labels/val/')
        
    return True

split_train_testset ()

True

Train Model

In [ ]:
%cd ../yolov5
!ls

/content/yolov5
data	    labels.png	 requirements.txt    test_batch0_pred.jpg  weights
detect.py   LICENSE	 results.png	     test.py
Dockerfile  models	 results.txt	     train.py
hubconf.py  __pycache__  runs		     tutorial.ipynb
inference   README.md	 test_batch0_gt.jpg  utils


In [48]:
!python train.py --img 512 --batch 24 --epochs 10 --data ./data/wheat.yaml --cfg ./models/yolov5l.yaml --weights ./weights/yolov5l.pt --single-cls --resume

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=24, bucket='', cache_images=False, cfg='./models/yolov5l.yaml', data='./data/wheat.yaml', device='', epochs=10, evolve=False, img_size=[512], multi_scale=False, name='', noautoanchor=False, nosave=False, notest=False, rect=False, resume=True, single_cls=True, weights='weights/last.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-06-28 04:40:39.833196: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/

  

In [ ]:
!python detect.py --img 512 --iou-thres 0.7 --weights weights/best.pt --source ../wheat/images/test  --save-txt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', fourcc='mp4v', img_size=512, iou_thres=0.6, output='inference/output', save_txt=True, source='../wheat/images/test', view_img=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

image 1/10 ../wheat/images/test/2fd875eaa.jpg: 512x512 27 wheats, Done. (0.025s)
image 2/10 ../wheat/images/test/348a992bb.jpg: 512x512 37 wheats, Done. (0.027s)
image 3/10 ../wheat/images/test/51b3e36ab.jpg: 512x512 25 wheats, Done. (0.025s)
image 4/10 ../wheat/images/test/51f1be19e.jpg: 512x512 18 wheats, Done. (0.024s)
image 5/10 ../wheat/images/test/53f253011.jpg: 512x512 29 wheats, Done. (0.027s)
image 6/10 ../wheat/images/test/796707dd7.jpg: 512x512 26 wheats, Done. (0.024s)
image 7/10 ../wheat/images/test/aac893a91.jpg: 512x512 20 wheats, Done. (0.021s)
image 8/10 ../wheat/images/test/cb8d261a3.jpg: 512x512 26 wheats, Done. (0.021s)
image 9/10 ../wheat/image

Save Model to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [49]:
%cp /content/yolov5/weights/best.pt  "/content/gdrive/My Drive/Colab Notebooks/WheatDetection/"
#%cp /content/yolov5/weights/last.pt  "/content/gdrive/My Drive/Colab Notebooks/WheatDetection/"

Opt Reload Model from Drive

In [ ]:
%cp "/content/gdrive/My Drive/Colab Notebooks/WheatDetection/last.pt" /content/yolov5/weights

In [ ]:
%cd ..

/content


In [ ]:
shutil.make_archive('output', 'zip', 'inference/output')
#shutil.make_archive('wheat-processed', 'zip', 'wheat')
#%cp /content/wheat-processed.zip "/content/gdrive/My Drive/Colab Notebooks/WheatDetection/"